<a href="https://colab.research.google.com/github/Winindu/ML_CW/blob/master/Copy_of_telco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, random
import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Telco Customers.csv")

df.drop_duplicates(inplace=True)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")
df["TotalCharges"].fillna(df["MonthlyCharges"] * df["tenure"], inplace=True)

if "customerID" in df.columns:
    df.drop(columns=["customerID"], inplace=True)

print(df.head())
print(df.shape)


In [ ]:
print("\n=== Dataset Shape ===")
print(df.shape)

print("\n=== First 5 Rows ===")
display(df.head())

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics (Numerical) ===")
display(df.describe())

print("\n=== Summary Statistics (Categorical) ===")
display(df.describe(include=['object']))
